In [1]:
import sqlite3

In [2]:
sqlite3.sqlite_version

'3.41.2'

In [3]:
conn = sqlite3.connect('contact.db')

In [4]:
cursor = conn.cursor()
print(cursor)

In [5]:
cursor.execute('DROP TABLE contact')

OperationalError: no such table: contact

In [6]:
cursor.execute("""CREATE TABLE contact(
    name text,
    age int,
    email text)
""")

In [8]:
cursor.execute("INSERT INTO contact VALUES('kim', 30, 'kim@naver.com')")

In [11]:
cursor.execute("INSERT INTO contact VALUES(?,?,?)", ("lee",30,"lee@daum.net"))

In [12]:
cursor.execute("INSERT INTO contact VALUES(:name, :age, :email)", 
               {"name":"heo", "age":30, "email":"heojk@daum.net"})

위 문법 모두 같은 cursor객체(0x1e7a02791c0)를 반환한다.

In [13]:
conn.commit()

In [14]:
conn.close()

# 데이터 읽기

In [15]:
conn = sqlite3.connect('contact.db')

In [16]:
cursor = conn.cursor()

In [17]:
cursor.execute('select * from contact')

In [18]:
for row in cursor:
    print(row)

('kim', 30, 'kim@naver.com')
('lee', 30, 'lee@daum.net')
('heo', 30, 'heojk@daum.net')


In [20]:
!conda install cx_Oracle

^C


In [21]:
import cx_Oracle

In [28]:
dsn = cx_Oracle.makedsn('localhost', 1521, sid='xe')
print(dsn)

(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))


In [25]:
conn = cx_Oracle.connect(user='hr', password='hr', dsn=dsn)
print(conn)

DatabaseError: DPI-1047: Cannot locate a 64-bit Oracle Client library: "The specified module could not be found". See https://cx-oracle.readthedocs.io/en/latest/user_guide/installation.html for help

In [29]:
import oracledb as db

In [30]:
# conn = db.connect(dsn='localhost:1521/xe', user='hr', password='hr')
conn = db.connect(dsn=f"hr/hr@localhost:1521/xe")
print(conn)

<oracledb.Connection to hr@localhost:1521/xe>


In [32]:
cursor = conn.cursor()
print(cursor)

<oracledb.Cursor on <oracledb.Connection to hr@localhost:1521/xe>>


In [33]:
cursor.execute('select * from employees')

<oracledb.Cursor on <oracledb.Connection to hr@localhost:1521/xe>>

In [34]:
print(cursor.fetchone())

(100, 'Steven', 'King', 'SKING', '1.515.555.0100', datetime.datetime(2013, 6, 17, 0, 0), 'AD_PRES', 24000.0, None, None, 90)


In [35]:
cursor.description

[('EMPLOYEE_ID', <DbType DB_TYPE_NUMBER>, 7, None, 6, 0, False),
 ('FIRST_NAME', <DbType DB_TYPE_VARCHAR>, 20, 20, None, None, True),
 ('LAST_NAME', <DbType DB_TYPE_VARCHAR>, 25, 25, None, None, False),
 ('EMAIL', <DbType DB_TYPE_VARCHAR>, 25, 25, None, None, False),
 ('PHONE_NUMBER', <DbType DB_TYPE_VARCHAR>, 20, 20, None, None, True),
 ('HIRE_DATE', <DbType DB_TYPE_DATE>, 23, None, None, None, False),
 ('JOB_ID', <DbType DB_TYPE_VARCHAR>, 10, 10, None, None, False),
 ('SALARY', <DbType DB_TYPE_NUMBER>, 12, None, 8, 2, True),
 ('COMMISSION_PCT', <DbType DB_TYPE_NUMBER>, 6, None, 2, 2, True),
 ('MANAGER_ID', <DbType DB_TYPE_NUMBER>, 7, None, 6, 0, True),
 ('DEPARTMENT_ID', <DbType DB_TYPE_NUMBER>, 5, None, 4, 0, True)]

# 회원관리 시스템 예제

In [55]:
cursor.execute('drop table member')

In [56]:
cursor.execute('CREATE TABLE member (\
name VARCHAR2(20), \
phone VARCHAR2(20), \
email VARCHAR2(50), \
age NUMBER(3), \
grade NUMBER(2), \
etc VARCHAR2(100))')

In [57]:
class Member:
    def __init__(self, name, phone, email, age, grade, etc):
        self.name = name
        self.phone = phone
        self.email = email
        self.age = age
        self.grade = grade
        self.etc = etc

    def __str__(self):
        return (f"name: {self.name}, phone: {self.name},"
                f"email: {self.email}, age: {self.age},"
                f"grade: {self.grade}, etc: {self.etc}")

    def to_dict(self):
        return {"name":self.name, "phone":self.phone, "email":self.email,
                "age":self.age, "grade":self.grade, "etc":self.etc}

In [58]:
def insert_member_info():
    cursor = conn.cursor()
    name = input("이름: ")
    phone = input("전화번호: ")
    email = input("이메일: ")
    age = int(input("나이: "))
    grade = int(input("고객등급(1~5): "))
    etc = input("기타정보: ")
    member = Member(name, phone, email, age, grade, etc)
    sql = """insert into member
            values (:name, :phone, :email, :age, :grade, :etc)"""
    cursor.execute(sql, member.to_dict())

In [59]:
def get_all_members():
    cursor=conn.cursor()
    cursor.execute('select * from member')
    for data in cursor:
        print(Member(*data)) # *언패킹

In [60]:
def search_member(name):
    cursor = conn.cursor()
    sql = 'select * from member where name=:name'
    cursor.execute(sql, (name,))
    for data in cursor:
        print(Member(*data))

In [61]:
def delete_member(email):
    cursor = conn.cursor()
    sql = 'delete from member where email=:email'
    cursor.execute(sql, (email,))

In [62]:
def export_member():
    file_name = input('파일명을 입력하세요.')
    cursor = conn.cursor()
    cursor.execute('select * from member')
    members = cursor.fetchall()
    colnames = [row[0] for row in cursor.description ]
    import csv
    with open(file_name, 'w', newline='', encoding='UTF-8-SIG') as file:
        w = csv.writer(file, quoting=csv.QUOTE_NONNUMERIC)
        w.writerow(colnames)
        w.writerows(members)

In [63]:
def update_member():
    cursor = conn.cursor()
    email = input('수정할 사원의 이메일을 입력하세요. ')
    sql1 = 'select * from member where email=:email'
    cursor.execute(sql1, (email,))
    data = cursor.fetchone()
    m = Member(*data)
    name = input(f"이름({m.name}) : ")
    phone = input(f"전화번호({m.phone}) : ")
    age = input(f"나이({m.age}) : ")
    grade = input(f"등급({m.grade}) : ")
    etc = input(f"기타정보({m.etc}) : ")
    sql = """update member set name=:name, phone=:phone, age=:age, grade=:grade, etc=:etc where email=:email"""
    cursor.execute(sql, (name, phone, age, grade, etc, email))

In [64]:
def print_menu():
    print("1.입력", "2.전체조회", "3.찾기", "4.삭제", "5.내보내기(CSV)", "6.수정", "0.종료", sep=" | ", end="")
    menu = input("메뉴선택: ")
    return int(menu)

In [65]:
def main():
    while True:
        menu = print_menu()
        if menu==1:
            insert_member_info()
            conn.commit()
        elif menu==2:
            get_all_members()
        elif menu==3:
            name = input('찾고 싶은 회원의 이름을 입력하세요.')
            search_member(name)
        elif menu==4:
            email = input('삭제할 회원의 이메일을 입력하세요.')
            delete_member(email)
            conn.commit()
        elif menu==5:
            export_member()
        elif menu==6:
            update_member()
            conn.commit()
        elif menu==0:
            conn.close()
            break

In [66]:
if __name__ == '__main__':
    import oracledb as db
    global conn
    # oracle_dsn = db.makedsn(host='localhost', port=1521, sid='xe')
    # conn = db.connect(dsn=oracle_dsn)
    conn = db.connect(dsn=f"hr/hr@localhost:1521/xe")
    main()

1.입력 | 2.전체조회 | 3.찾기 | 4.삭제 | 5.내보내기(CSV) | 6.수정 | 0.종료

메뉴선택:  1
이름:  홍길동
전화번호:  010-222-3333
이메일:  hong@hong.com
나이:  25
고객등급(1~5):  4
기타정보:  nothing


1.입력 | 2.전체조회 | 3.찾기 | 4.삭제 | 5.내보내기(CSV) | 6.수정 | 0.종료

메뉴선택:  1
이름:  홍길서
전화번호:  010-444-5555
이메일:  kilseo@hong.com
나이:  23
고객등급(1~5):  3
기타정보:  nothing


1.입력 | 2.전체조회 | 3.찾기 | 4.삭제 | 5.내보내기(CSV) | 6.수정 | 0.종료

...

메뉴선택:  2


name: 홍길길동, phone: 홍길길동,email: hong@hong.com, age: 26,grade: 4, etc: nothing
name: 홍길서, phone: 홍길서,email: kilseo@hong.com, age: 23,grade: 3, etc: nothing
1.입력 | 2.전체조회 | 3.찾기 | 4.삭제 | 5.내보내기(CSV) | 6.수정 | 0.종료

메뉴선택:  5
파일명을 입력하세요. memberms2.csv


1.입력 | 2.전체조회 | 3.찾기 | 4.삭제 | 5.내보내기(CSV) | 6.수정 | 0.종료

메뉴선택:  0
